# Lemma dilemma

*Creator: Magnus Struckmann*
## Table of contents

- Notebook summary
- Create data sets

### Notebook summary

Based on the program developed in the lecture (SparseRandomProjections), analyze a database of your choice (but not exactly the same digits data as in the lecture) using random projections.
Study the accuracy (or a score of your choice that makes most sense for your data) as a function of the number of dimensions / features that survived the random projection.

In [ ]:
# Import packages
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from sklearn import datasets

### Create data sets

In [ ]:
data_collection = []
